In [61]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

In [62]:
ROOT = Path.cwd().resolve()
DATA_FILE = next(ROOT.rglob("extracted_data/timeline_visualisation.xlsx"))

data = pd.read_excel(DATA_FILE)



In [63]:
class Timeline:
    def __init__(self, data: pd.DataFrame):
        df = data.sort_values('authors', ascending=False).reset_index(drop=True)
        self.authors = df['authors'].tolist()
        self.timepoints = df['time_points'].tolist()
        self.ylist = list(range(4, 4*len(self.authors)+4, 4))

        n = max(1, len(self.authors))
        cmap = plt.get_cmap('viridis')
        self.colors = [cmap(i/(n-1) if n > 1 else 0.5) for i in range(n)]

    def _tokens(self, s):
        if s is None or (isinstance(s, float) and np.isnan(s)) or (isinstance(s, pd.Series) and s.isna().all()):
            return []
        s = str(s).strip()
        if not s:
            return []
        return [t.strip() for t in s.split(',') if t.strip() and t.strip().lower() != 'nan']

    def _parse(self, token: str):
        if ':' in token:
            a, b = token.split(':', 1)
            try:
                a, b = float(a), float(b)
            except ValueError:
                return None
            if b < a:
                a, b = b, a
            return ("range", a, b)
        else:
            try:
                return ("point", float(token))
            except ValueError:
                return None

    def plot_line(self, timepoints, y, color):
        parsed = [p for t in self._tokens(timepoints) if (p := self._parse(t)) is not None]
        xs = []
        for kind, *vals in parsed:
            xs.extend(vals)
        if xs:
            plt.plot([min(xs), max(xs)], [y, y], '-', linewidth=1,
                     solid_capstyle='round', color=color)
        pts = []
        for kind, *vals in parsed:
            if kind == "range":
                a, b = vals
                plt.plot([a, b], [y, y], '-', linewidth=6,
                         solid_capstyle='round', color=color, alpha=0.5)
            else:
                pts.append(vals[0])
        if pts:
            plt.scatter(pts, [y]*len(pts), color=color)

    def set_yticks(self):
        plt.yticks(self.ylist, self.authors)

    def parse_citations(self, citations):
        parsed_citations = []
        for citation in citations:
            name_match = re.match(r'^([A-Za-z\\s,.\\-]+?)\\s*,', citation)
            year_match = re.search(r'\\((\\d{4})\\)', citation)
            name = name_match.group(1) if name_match else ''
            year = year_match.group(1) if year_match else ''
            result = f"{name.strip()} ({year})"
            parsed_citations.append(result)
        return parsed_citations

    def show(self):
        plt.figure(figsize=(10, 30))
        for tp, y, color in zip(self.timepoints, self.ylist, self.colors):
            self.plot_line(tp, y, color)
        plt.yticks(self.ylist, self.authors)
        plt.axvline(x=0, color='black', linestyle='-')
        plt.title('Measurement points of included studies')
        plt.xlabel('Time since trauma (months)')
        plt.tight_layout()
        ax = plt.gca()
        ax.margins(y=0.01)
        plt.show()

In [64]:
timeline = Timeline(data)
timeline.show()